In [74]:
import numpy as np
import re

In [ ]:
def read_file():
    while True:
        try:
            name_file = input("Enter a class file to grade (e.g., class1 for class1.txt): ")
            with open(f"./Data_Files/input/{name_file}.txt", 'r') as file:
                file_read = file.read().splitlines()
            print(f"Successfully opened {name_file}.txt")
            return file_read, name_file
        except FileNotFoundError:
            print("File cannot be found.")

In [76]:
file_read, name_file = read_file()

Successfully opened class8.txt


In [77]:
def analyzing_data(file_read):
    print("****ANALYZING****")
    count = len(file_read)
    error_count = 0
    correct_exams = []

    for line in file_read:
        res = re.findall(r"[N]\d{8}", line)
        if not res:
            error_count += 1
            print(f"Invalid line of data: N# is invalid \n {line}")
        else:
            element = line.split(",")
            if len(element) != 26:
                error_count += 1
                print(f"Invalid line of data: does not contain exactly 26 values:\n {line}")
            else:
                correct_exams.append(line)

    if error_count == 0:
        print("No error found!")

    print("****REPORT****")
    print(f"Total valid lines of data:", count - error_count)
    print(f"Total invalid lines of data:", error_count)
    
    return correct_exams

In [78]:
correct_exams = analyzing_data(file_read)

****ANALYZING****
No error found!
****REPORT****
Total valid lines of data: 49
Total invalid lines of data: 0


In [79]:
answer_key = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
def calculate_scores(correct_exams, answer_key):
    lst_of_scores = []
    skip_questions = {}
    wrong_questions = {}

    keys = answer_key.split(",")
    for exam in correct_exams:
        result = exam.split(",")
        score = 0
        for idx, answer in enumerate(result[1:], start=1):
            if answer == '':
                skip_questions[idx] = skip_questions.get(idx, 0) + 1
            else:
                if answer == keys[idx - 1]:
                    score += 4
                else:
                    score -= 1
                    wrong_questions[idx] = wrong_questions.get(idx, 0) + 1
        lst_of_scores.append(score)

    return lst_of_scores, skip_questions, wrong_questions
lst_of_scores, skip_questions, wrong_questions = calculate_scores(correct_exams, answer_key)

In [80]:
def statistic_scores(lst_of_scores, skip_questions, wrong_questions):
    print("Total students with high scores:", sum(score > 80 for score in lst_of_scores))
    print("Mean (average) score:", np.mean(lst_of_scores))
    print("Highest score:", max(lst_of_scores))
    print("Lowest score:", min(lst_of_scores))
    print("Range of scores:", max(lst_of_scores) - min(lst_of_scores))
    print("Median score:", np.median(lst_of_scores))

    most_skip_values = max(skip_questions.values())
    most_wrong_values = max(wrong_questions.values())

    most_skip_keys = [key for key, value in skip_questions.items() if value == most_skip_values]
    print("Question that most people skip:")
    for question in sorted(most_skip_keys):
        print(f"{question} - {most_skip_values} - {most_skip_values / 25}")

    most_wrong_keys = [key for key, value in wrong_questions.items() if value == most_wrong_values]
    print("Question that most people answer incorrectly:")
    for question in sorted(most_wrong_keys):
        print(f"{question} - {most_wrong_values} - {most_wrong_values / 25}")

In [81]:
statistic_scores(lst_of_scores, skip_questions, wrong_questions)

Total students with high scores: 17
Mean (average) score: 76.6734693877551
Highest score: 92
Lowest score: 59
Range of scores: 33
Median score: 77.0
Question that most people skip:
4 - 9 - 0.36
Question that most people answer incorrectly:
4 - 9 - 0.36
15 - 9 - 0.36


In [82]:
def update_scores(correct_exams, lst_of_scores, name_file):
    res = []
    for exam, score in zip(correct_exams, lst_of_scores):
        res.append([exam.split(",")[0], score])

    with open(f"Data_Files/output/{name_file}_grades.txt", 'a') as f:
        for element in res:
            f.write(",".join(map(str, element)) + "\n")

update_scores(correct_exams, lst_of_scores, name_file)